In [41]:
#@title ##Import libraries and tools.
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import pandas

from PIL import Image
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tqdm import tqdm

!sudo apt-get install webp parallel
!git clone https://github.com/sahal-mulki/SpottingDiffusion/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
webp is already the newest version (1.2.2-2ubuntu0.22.04.2).
Suggested packages:
  ash csh fish ksh tcsh zsh isag
The following NEW packages will be installed:
  parallel sysstat
0 upgraded, 2 newly installed, 0 to remove and 32 not upgraded.
Need to get 2,434 kB of archives.
After this operation, 4,521 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 sysstat amd64 12.5.2-2ubuntu0.2 [487 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 parallel all 20210822+ds-2 [1,947 kB]
Fetched 2,434 kB in 2s (1,607 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (

In [7]:
#@markdown ##Downloading the SpottingDiffusion Testing dataset.
#@markdown <br>

KaggleAPIUsername = "sahalmulki" #@param {type:"string"}
KaggleAPIKey = "c0488ed13ac0db114a3350bdb5778e73" #@param {type:"string"}

os.environ['KAGGLE_USERNAME'] = KaggleAPIUsername
os.environ['KAGGLE_KEY'] = KaggleAPIKey

!kaggle datasets download sahalmulki/spotting-diffusion-testing-balanced
!mkdir data
!unzip /content/spotting-diffusion-testing-balanced.zip -d /content/spotting-diffusion-testing-balanced/

spotting-diffusion-testing-balanced.zip: Skipping, found more recently modified local copy (use --force to force download)
mkdir: cannot create directory ‘data’: File exists
Archive:  /content/spotting-diffusion-testing-balanced.zip
replace /content/spotting-diffusion-testing-balanced/spotting-diffusion-testing-balanced/ldms/r-dalle2/15mm-wide-angle-lens-photo-of-a-rapper-in-1990-new-york-v0-rgr4pg3ka9791.webp? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [57]:
#@markdown ##Processing the dataset.
#@markdown <br>

!mkdir spottingdiffusion-testing-dataset-processed && cd spottingdiffusion-testing-dataset-processed && mkdir Real && cd Real && mkdir Real && cp /content/spotting-diffusion-testing-balanced/spotting-diffusion-testing-balanced/real/r-pics/* /content/spottingdiffusion-testing-dataset-processed/Real/Real
!cd spottingdiffusion-testing-dataset-processed && mkdir Generated && cd Generated && mkdir Generated && cd Generated && cp /content/spotting-diffusion-testing-balanced/spotting-diffusion-testing-balanced/ldms/r-dalle2/* /content/spottingdiffusion-testing-dataset-processed/Generated/Generated/ && cp /content/spotting-diffusion-testing-balanced/spotting-diffusion-testing-balanced/ldms/r-midjourney/* /content/spottingdiffusion-testing-dataset-processed/Generated/Generated/ && cp /content/spotting-diffusion-testing-balanced/spotting-diffusion-testing-balanced/ldms/r-stablediffusion/* /content/spottingdiffusion-testing-dataset-processed/Generated/Generated/

!find . -name "*.webp" -print0 | parallel -0 dwebp -quiet {} -o {.}.png


In [58]:
#@markdown ##Evaluating the model on the dataset.
#@markdown <br>

!python /content/SpottingDiffusion/evaluate.py /content/spottingdiffusion-testing-dataset-processed /content/SpottingDiffusion/pretrained/pretrained-spotting-diffusion

2024-02-11 16:53:47.398216: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-11 16:53:47.398262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-11 16:53:47.399514: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-11 16:53:48.354539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-02-11 16:53:50.120339: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Found 120 files belonging